# 🎯 제스처 인식 TinyML 학습 파이프라인 (개선된 INT8 양자화)

이 노트북은 드럼 히트(Drum_hit), 피아노(piano), 바이올린(violin) 제스처를 인식하기 위한 TinyML 모델 학습 파이프라인을 구현합니다.

## 목차
1. 데이터 분석 및 탐색
2. 데이터 정제 및 전처리
3. 데이터 증강 및 시각화
4. 모델 설계 및 학습
5. 모델 평가 및 TFLite 변환
6. **🚀 개선된 INT8 양자화 및 성능 점검** ⭐ 대폭 개선
7. Arduino 배포 가이드

---

## 1. 데이터 분석 및 탐색

먼저 필요한 라이브러리를 로드하고 제스처 데이터를 분석합니다.

In [ ]:
# 필요한 라이브러리 설치 (처음 실행 시 필요)
!pip install scikit-learn tensorflow pandas matplotlib seaborn

# 양자화 인식 훈련을 위한 라이브러리 (선택사항)
# 설치 오류 시 주석 처리하고 진행하세요
# !pip install tensorflow-model-optimization

In [1]:
# 기본 라이브러리 로드
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import glob
import time
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 양자화 인식 훈련 라이브러리 (선택사항)
try:
    import tensorflow_model_optimization as tfmot
    QAT_AVAILABLE = True
    print("✅ 양자화 인식 훈련(QAT) 사용 가능")
except ImportError:
    QAT_AVAILABLE = False
    print("⚠️ 양자화 인식 훈련(QAT) 라이브러리 없음 - 다른 방법으로 진행")

# 시드 고정 (재현성 확보)
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)
print(f"TensorFlow 버전: {tf.__version__}")

# 데이터 경로 설정 - 사용자 지정 경로
DATA_ROOT = "C:/Users/user/Desktop/tinymlmagicwand"  # 사용자 지정 경로
print(f"데이터 루트 경로: {DATA_ROOT}")

⚠️ 양자화 인식 훈련(QAT) 라이브러리 없음 - 다른 방법으로 진행
TensorFlow 버전: 2.5.0
데이터 루트 경로: C:/Users/user/Desktop/tinymlmagicwand


In [2]:
# 제스처 폴더 정의
gesture_folders = {
    '날다': os.path.join(DATA_ROOT, "up"),
    '앉다': os.path.join(DATA_ROOT, "down"),
    '돌다': os.path.join(DATA_ROOT, "turn")
}

# 각 폴더의 CSV 파일 목록 확인
for gesture_name, folder_path in gesture_folders.items():
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    print(f"\n{gesture_name} 제스처 데이터 폴더: {folder_path}")
    print(f"CSV 파일 수: {len(csv_files)}")
    print(f"파일 예시: {os.path.basename(csv_files[0]) if csv_files else '없음'}")


날다 제스처 데이터 폴더: C:/Users/user/Desktop/tinymlmagicwand\up
CSV 파일 수: 53
파일 예시: gesture_001.csv

앉다 제스처 데이터 폴더: C:/Users/user/Desktop/tinymlmagicwand\down
CSV 파일 수: 50
파일 예시: gesture_001.csv

돌다 제스처 데이터 폴더: C:/Users/user/Desktop/tinymlmagicwand\turn
CSV 파일 수: 51
파일 예시: gesture_001.csv


In [3]:
# 각 제스처 폴더에서 첫 번째 CSV 파일을 샘플로 분석
for gesture_name, folder_path in gesture_folders.items():
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    if csv_files:
        sample_file = csv_files[0]
        df = pd.read_csv(sample_file, skiprows=1)
        print(f"\n{gesture_name} 제스처 데이터 샘플 (파일: {os.path.basename(sample_file)})")
        print(f"데이터 형태: {df.shape}")
        print("컬럼명:", df.columns.tolist())
        print("샘플 데이터:")
        print(df.head(3))
        
        # 기본 통계 정보
        print(f"\n{gesture_name} 기본 통계:")
        print(df.describe().round(3))


날다 제스처 데이터 샘플 (파일: gesture_001.csv)
데이터 형태: (120, 6)
컬럼명: ['aX', 'aY', 'aZ', 'gX', 'gY', 'gZ']
샘플 데이터:
       aX     aY     aZ      gX      gY      gZ
0  -0.344 -0.114  0.935  -3.906 -73.059 -21.362
1  -0.351 -0.130  0.945   5.310 -72.754 -16.846
2  -0.355 -0.163  0.911  13.672 -71.350 -12.451

날다 기본 통계:
            aY       aZ       gX       gY       gZ
count  119.000  119.000  119.000  119.000  119.000
mean     0.294    0.025   15.962   96.656   16.623
std      0.380    1.424   45.732  192.079   74.554
min     -0.522   -2.060  -85.144 -284.363 -146.057
25%      0.048   -1.046  -17.486  -55.939  -20.691
50%      0.295   -0.302   13.855   87.891   22.461
75%      0.543    0.923   51.972  289.032   77.148
max      1.038    3.651  127.258  363.342  119.202

앉다 제스처 데이터 샘플 (파일: gesture_001.csv)
데이터 형태: (120, 6)
컬럼명: ['aX', 'aY', 'aZ', 'gX', 'gY', 'gZ']
샘플 데이터:
      aX     aY     aZ     gX      gY     gZ
0  0.732  0.186  0.594  3.601  14.709 -4.150
1  0.730  0.181  0.554  5.005  15.076 -5

## 2. 데이터 정제 및 전처리

데이터 정제 및 전처리 단계에서는 다음 작업을 수행합니다:
1. 결측치 처리
2. 이상치 제거
3. 데이터 정규화
4. 레이블 인코딩

In [4]:
def load_and_preprocess_data(file_path, label, skip_rows=1):
    """
    CSV 파일에서 제스처 데이터를 로드하고 전처리합니다.
    
    Args:
        file_path (str): CSV 파일 경로
        label (int): 제스처 레이블 (0: 날다, 1: 앉다, 2: 돌다)
        skip_rows (int): 건너뛸 헤더 행 수
        
    Returns:
        tuple: (전처리된 데이터, 레이블)
    """
    try:
        # 데이터 로드
        df = pd.read_csv(file_path, skiprows=skip_rows)
        
        # 컬럼명 공백 제거
        df.columns = df.columns.str.strip()
        
        # 숫자로 변환 불가능한 행 제거
        df = df[pd.to_numeric(df['aX'], errors='coerce').notna()]
        df = df.astype(float)
        
        # 결측치 처리 (앞뒤 값의 평균으로 대체)
        df = df.interpolate(method='linear', limit_direction='both')
        
        # 이상치 처리 (3시그마 규칙 적용)
        for col in df.columns:
            mean = df[col].mean()
            std = df[col].std()
            df[col] = df[col].clip(mean - 3*std, mean + 3*std)
        
        # 원-핫 인코딩된 레이블 생성
        one_hot_label = np.zeros(3)
        one_hot_label[label] = 1
        
        return df, one_hot_label
    
    except Exception as e:
        print(f"오류 발생 (파일: {file_path}): {e}")
        return None, None

In [5]:
def normalize_data(df):
    """
    가속도계와 자이로스코프 데이터를 정규화합니다.
    
    Args:
        df (DataFrame): 원본 데이터프레임
        
    Returns:
        DataFrame: 정규화된 데이터프레임
    """
    normalized_df = df.copy()
    
    # 가속도계 데이터 정규화 (범위: -4 ~ 4)
    for col in ['aX', 'aY', 'aZ']:
        normalized_df[col] = (df[col] + 4) / 8
    
    # 자이로스코프 데이터 정규화 (범위: -2000 ~ 2000)
    for col in ['gX', 'gY', 'gZ']:
        normalized_df[col] = (df[col] + 2000) / 4000
    
    return normalized_df

## 3. 데이터 증강 및 시각화

데이터 증강을 통해 학습 데이터의 양을 늘리고 모델의 일반화 성능을 향상시킵니다. 다음과 같은 증강 기법을 적용합니다:
1. 노이즈 추가
2. 시간 이동
3. 스케일 변화

In [6]:
def augment_data(df, noise_factor=0.05, time_shift_max=5):
    """
    데이터 증강을 통해 추가 샘플을 생성합니다.
    
    Args:
        df (DataFrame): 원본 데이터프레임
        noise_factor (float): 노이즈 강도
        time_shift_max (int): 최대 시간 이동 범위
        
    Returns:
        DataFrame: 증강된 데이터프레임
    """
    augmented_dfs = []
    
    # 원본 데이터 추가
    augmented_dfs.append(df.copy())
    
    # 노이즈 추가
    noise_df = df.copy()
    for col in df.columns:
        noise = np.random.normal(0, noise_factor, size=len(df))
        noise_df[col] = noise_df[col] + noise
    augmented_dfs.append(noise_df)
    
    # 시간 이동
    time_shift = np.random.randint(1, time_shift_max + 1)
    time_shift_df = df.copy()
    time_shift_df = pd.concat([time_shift_df.iloc[time_shift:], time_shift_df.iloc[:time_shift]])
    time_shift_df.reset_index(drop=True, inplace=True)
    augmented_dfs.append(time_shift_df)
    
    # 스케일 변화
    scale_factor = np.random.uniform(0.9, 1.1)
    scale_df = df.copy()
    for col in df.columns:
        scale_df[col] = scale_df[col] * scale_factor
    augmented_dfs.append(scale_df)
    
    # 모든 증강 데이터 결합
    result_df = pd.concat(augmented_dfs, axis=0)
    result_df.reset_index(drop=True, inplace=True)
    
    return result_df

## 4. 데이터 통합 및 모델 학습 준비

모든 제스처 데이터를 통합하고 모델 학습을 위한 형태로 변환합니다.

In [7]:
def process_all_data():
    """
    모든 제스처 데이터를 처리하고 저장합니다.
    
    Returns:
        tuple: (입력 데이터, 출력 레이블)
    """
    # 데이터 폴더 정의
    data_folders = [
        (gesture_folders['날다'], 0, '날다'),
        (gesture_folders['앉다'], 1, '앉다'),
        (gesture_folders['돌다'], 2, '돌다')
    ]
    
    # 결과 저장 디렉토리 생성 (사용자 지정 경로)
    output_dir = os.path.join(DATA_ROOT, 'processed_data')
    os.makedirs(output_dir, exist_ok=True)
    
    all_data = []
    all_labels = []
    
    for folder_path, label, gesture_name in data_folders:
        print(f"\n{gesture_name} 제스처 데이터 처리 중...")
        csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
        print(f"발견된 CSV 파일 수: {len(csv_files)}")
        
        for file_path in csv_files:
            df, one_hot_label = load_and_preprocess_data(file_path, label)
            
            if df is not None:
                # 데이터 정규화
                normalized_df = normalize_data(df)
                
                # 데이터 길이 맞추기 (119개 행으로 고정)
                if len(normalized_df) > 119:
                    normalized_df = normalized_df.iloc[:119]
                elif len(normalized_df) < 119:
                    # 패딩 (마지막 값으로 채우기)
                    last_row = normalized_df.iloc[-1]
                    padding_rows = 119 - len(normalized_df)
                    padding_df = pd.DataFrame([last_row] * padding_rows)
                    normalized_df = pd.concat([normalized_df, padding_df], ignore_index=True)
                
                all_data.append(normalized_df.values)
                all_labels.append(one_hot_label)
    
    # NumPy 배열로 변환
    X = np.array(all_data)
    y = np.array(all_labels)
    
    print(f"\n처리된 데이터 형태: {X.shape}, {y.shape}")
    
    return X, y

# 데이터 처리 실행
X, y = process_all_data()


날다 제스처 데이터 처리 중...
발견된 CSV 파일 수: 53

앉다 제스처 데이터 처리 중...
발견된 CSV 파일 수: 50

돌다 제스처 데이터 처리 중...
발견된 CSV 파일 수: 51

처리된 데이터 형태: (154, 119, 6), (154, 3)


In [8]:
# sklearn 없이 직접 구현한 train_test_split
def custom_train_test_split(X, y, test_size=0.2, val_size=0.1, random_state=None):
    """
    sklearn 없이 직접 구현한 데이터 분할 함수
    """
    if random_state is not None:
        np.random.seed(random_state)
    
    n_samples = len(X)
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    # 테스트 세트 분할
    test_split = int(n_samples * test_size)
    test_indices = indices[:test_split]
    remaining_indices = indices[test_split:]
    
    # 검증 세트 분할
    val_split = int(len(remaining_indices) * val_size / (1 - test_size))
    val_indices = remaining_indices[:val_split]
    train_indices = remaining_indices[val_split:]
    
    return (X[train_indices], X[val_indices], X[test_indices],
            y[train_indices], y[val_indices], y[test_indices])

# 데이터 분할
inputs_train, inputs_val, inputs_test, outputs_train, outputs_val, outputs_test = custom_train_test_split(
    X, y, test_size=0.15, val_size=0.15, random_state=SEED
)

print(f"훈련 데이터: {inputs_train.shape}")
print(f"검증 데이터: {inputs_val.shape}")
print(f"테스트 데이터: {inputs_test.shape}")

훈련 데이터: (108, 119, 6)
검증 데이터: (23, 119, 6)
테스트 데이터: (23, 119, 6)


## 5. 🚀 개선된 모델 설계 및 학습

양자화에 최적화된 TinyML 모델을 설계하고 학습합니다.

In [37]:
def create_quantization_friendly_model(input_shape, num_classes):
    """
    양자화에 최적화된 모델 구조
    """
    model = tf.keras.Sequential([
        # 더 안정적인 구조로 변경
        tf.keras.layers.Conv1D(8, 3, activation='relu', input_shape=input_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv1D(16, 5, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv1D(32, 7, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling1D(),  # Flatten 대신 GAP
        tf.keras.layers.Dropout(0.3),
        
        # 더 작은 Dense 레이어
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

def create_ultra_simple_model(input_shape, num_classes):
    """
    양자화에 매우 안전한 초간단 모델
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(8, 3, activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling1D(4),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# 모델 생성
input_shape = (119, 6)  # (시퀀스 길이, 특성 수)
num_classes = 3

print("🔧 양자화 최적화 모델 생성 중...")
model = create_quantization_friendly_model(input_shape, num_classes)
model.summary()

print("\n🔧 초간단 모델도 준비...")
simple_model = create_ultra_simple_model(input_shape, num_classes)
simple_model.summary()

🔧 양자화 최적화 모델 생성 중...
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 117, 8)            152       
_________________________________________________________________
batch_normalization_9 (Batch (None, 117, 8)            32        
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 58, 8)             0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 58, 8)             0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 54, 16)            656       
_________________________________________________________________
batch_normalization_10 (Batc (None, 54, 16)            64        
_________________________________________________________________
max_pooling1d_10 (MaxPooling (Non

In [63]:
# 콜백 설정
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=20,
    restore_best_weights=True
)

# 모델 저장 경로 (사용자 지정 경로)
model_save_path = os.path.join(DATA_ROOT, 'best_model.keras')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    model_save_path,
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# 모델 학습
print("🚀 모델 학습 시작...")
EPOCHS = 200
BATCH_SIZE = 16

history = model.fit(
    inputs_train,
    outputs_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(inputs_val, outputs_val),
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

print("✅ 모델 학습 완료!")

🚀 모델 학습 시작...
Epoch 1/200
7/7 [==============================] - 0s 9ms/step - loss: 0.0320 - accuracy: 1.0000 - val_loss: 0.1175 - val_accuracy: 1.0000

Epoch 00001: val_accuracy improved from -inf to 1.00000, saving model to C:/Users/user/Desktop/tinymlmagicwand\best_model.keras
Epoch 2/200
7/7 [==============================] - 0s 6ms/step - loss: 0.0249 - accuracy: 0.9907 - val_loss: 0.0912 - val_accuracy: 1.0000

Epoch 00002: val_accuracy did not improve from 1.00000
Epoch 3/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0190 - accuracy: 1.0000 - val_loss: 0.0673 - val_accuracy: 1.0000

Epoch 00003: val_accuracy did not improve from 1.00000
Epoch 4/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0152 - accuracy: 1.0000 - val_loss: 0.0508 - val_accuracy: 1.0000

Epoch 00004: val_accuracy did not improve from 1.00000
Epoch 5/200
7/7 [==============================] - 0s 5ms/step - loss: 0.0155 - accuracy: 1.0000 - val_loss: 0.0389 - val_accuracy: 1

In [64]:
# 모델 평가
print("📊 모델 성능 평가...")
pred = model.predict(inputs_test)
pred_labels = np.argmax(pred, axis=1)
true_labels = np.argmax(outputs_test, axis=1)

float32_accuracy = np.mean(pred_labels == true_labels)
print(f"\n✅ Float32 모델 정확도: {float32_accuracy:.4f} ({float32_accuracy*100:.2f}%)")

print("\n분류 리포트:")
print(classification_report(true_labels, pred_labels, target_names=["날다", "앉다", "돌다"]))
print("\n혼동 행렬:")
print(confusion_matrix(true_labels, pred_labels))

📊 모델 성능 평가...

✅ Float32 모델 정확도: 1.0000 (100.00%)

분류 리포트:
              precision    recall  f1-score   support

          날다       1.00      1.00      1.00        12
          앉다       1.00      1.00      1.00         4
          돌다       1.00      1.00      1.00         7

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23


혼동 행렬:
[[12  0  0]
 [ 0  4  0]
 [ 0  0  7]]


## 6. 🚀 개선된 INT8 양자화 및 성능 점검

이 섹션에서는 여러 가지 개선된 양자화 기법을 시도하여 최적의 성능을 달성합니다.

In [65]:
# 저장 디렉토리 생성 (사용자 지정 경로)
output_dir = os.path.join(DATA_ROOT, 'tflite_models')
os.makedirs(output_dir, exist_ok=True)
print(f"모델 저장 디렉토리: {output_dir}")

def convert_to_mixed_precision(model, X_train, model_path):
    """
    혼합 정밀도 양자화 (입출력 Float32, 내부 INT8)
    """
    def representative_data_gen():
        print(f"대표 데이터 생성 중... ({min(200, len(X_train))}개 샘플)")
        for i in range(min(200, len(X_train))):
            yield [X_train[i:i+1].astype(np.float32)]
    
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen
    
    # 혼합 정밀도 설정 (입출력 Float32, 내부 INT8)
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS_INT8,
        tf.lite.OpsSet.TFLITE_BUILTINS
    ]
    # 입출력 타입 지정하지 않음 (Float32 유지)
    
    print("혼합 정밀도 양자화 변환 중...")
    tflite_model = converter.convert()
    
    with open(model_path, 'wb') as f:
        f.write(tflite_model)
    
    print(f"✅ 혼합 정밀도 모델 저장 완료: {model_path}")
    return tflite_model

def enhanced_int8_conversion(model, X_train, y_train, model_path):
    """
    개선된 INT8 양자화 변환 (클래스별 균등 샘플링)
    """
    def comprehensive_representative_data_gen():
        print("포괄적 대표 데이터 생성 중... (클래스별 균등 샘플링)")
        
        # 클래스별 균등 샘플링
        samples_per_class = 150  # 클래스당 150개
        for class_idx in range(3):
            class_mask = np.argmax(y_train, axis=1) == class_idx
            class_data = X_train[class_mask]
            
            if len(class_data) > 0:
                n_samples = min(samples_per_class, len(class_data))
                indices = np.random.choice(len(class_data), n_samples, replace=False)
                
                for idx in indices:
                    yield [class_data[idx:idx+1].astype(np.float32)]
    
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = comprehensive_representative_data_gen
    
    # 완전 INT8 설정
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    
    print("개선된 INT8 양자화 변환 중...")
    tflite_model = converter.convert()
    
    with open(model_path, 'wb') as f:
        f.write(tflite_model)
    
    print(f"✅ 개선된 INT8 모델 저장 완료: {model_path}")
    return tflite_model

def ultra_enhanced_int8_conversion(model, X_train, y_train, model_path):
    """
    최고 성능 INT8 양자화 변환 (전체 훈련 데이터 사용)
    """
    def ultra_representative_data_gen():
        print("최고 성능 대표 데이터 생성 중... (전체 훈련 데이터 사용)")
        
        # 전체 훈련 데이터 사용
        total_samples = len(X_train)
        indices = np.arange(total_samples)
        np.random.shuffle(indices)
        
        for idx in indices:
            yield [X_train[idx:idx+1].astype(np.float32)]
    
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = ultra_representative_data_gen
    
    # 완전 INT8 설정
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8
    
    print("최고 성능 INT8 양자화 변환 중...")
    tflite_model = converter.convert()
    
    with open(model_path, 'wb') as f:
        f.write(tflite_model)
    
    print(f"✅ 최고 성능 INT8 모델 저장 완료: {model_path}")
    return tflite_model

모델 저장 디렉토리: C:/Users/user/Desktop/tinymlmagicwand\tflite_models


In [66]:
def quick_model_check(model_path, X_test, y_test, model_name="모델"):
    """
    빠른 모델 성능 확인
    """
    print(f"\n🔍 {model_name} 빠른 점검")
    
    # 모델 크기
    size_kb = os.path.getsize(model_path) / 1024
    print(f"모델 크기: {size_kb:.1f}KB")
    
    # 모델 로드
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # 타입 확인
    input_type = input_details[0]['dtype']
    output_type = output_details[0]['dtype']
    print(f"입력 타입: {input_type}")
    print(f"출력 타입: {output_type}")
    
    # 정확도 측정
    correct = 0
    inference_times = []
    
    for i in range(len(X_test)):
        input_data = X_test[i:i+1].astype(input_type)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        
        # 추론 시간 측정
        start_time = time.time()
        interpreter.invoke()
        inference_time = time.time() - start_time
        inference_times.append(inference_time)
        
        output = interpreter.get_tensor(output_details[0]['index'])
        pred = np.argmax(output[0])
        true = np.argmax(y_test[i])
        
        if pred == true:
            correct += 1
    
    accuracy = correct / len(X_test)
    avg_inference_time = np.mean(inference_times) * 1000  # ms
    
    print(f"정확도: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"평균 추론 시간: {avg_inference_time:.2f}ms")
    
    # 양자화 타입 확인
    if input_type == np.int8 and output_type == np.int8:
        print("✅ 완전 INT8 양자화")
    elif input_type == np.float32 and output_type == np.float32:
        print("📊 Float32 (또는 혼합 정밀도)")
    else:
        print(f"🔄 혼합 타입: 입력({input_type}), 출력({output_type})")
    
    return {
        'accuracy': accuracy,
        'avg_inference_time_ms': avg_inference_time,
        'model_size_kb': size_kb,
        'input_type': input_type,
        'output_type': output_type
    }

In [68]:
# 🚀 단계별 양자화 성능 개선 실행
print("🎯 단계별 양자화 성능 개선 시작!")
print("="*60)

results_summary = {}

# Float32 기준 성능
print(f"\n📊 기준 성능 (Float32): {float32_accuracy*100:.2f}%")
results_summary['Float32'] = float32_accuracy

# 1단계: 혼합 정밀도 양자화
print("\n🔄 1단계: 혼합 정밀도 양자화")
mixed_precision_path = os.path.join(output_dir, 'mixed_precision.tflite')
mixed_model = convert_to_mixed_precision(model, inputs_train, mixed_precision_path)
mixed_results = quick_model_check(mixed_precision_path, inputs_test, outputs_test, "혼합 정밀도")
results_summary['혼합 정밀도'] = mixed_results['accuracy']

# 2단계: 개선된 INT8 양자화
print("\n🎯 2단계: 개선된 INT8 양자화 (클래스별 균등 샘플링)")
enhanced_int8_path = os.path.join(output_dir, 'enhanced_int8.tflite')
enhanced_model = enhanced_int8_conversion(model, inputs_train, outputs_train, enhanced_int8_path)
enhanced_results = quick_model_check(enhanced_int8_path, inputs_test, outputs_test, "개선된 INT8")
results_summary['개선된 INT8'] = enhanced_results['accuracy']

# 3단계: 최고 성능 INT8 양자화
print("\n🚀 3단계: 최고 성능 INT8 양자화 (전체 데이터 사용)")
ultra_int8_path = os.path.join(output_dir, 'ultra_int8.tflite')
ultra_model = ultra_enhanced_int8_conversion(model, inputs_train, outputs_train, ultra_int8_path)
ultra_results = quick_model_check(ultra_int8_path, inputs_test, outputs_test, "최고 성능 INT8")
results_summary['최고 성능 INT8'] = ultra_results['accuracy']

# 4단계: QAT 시도 (가능한 경우)
if QAT_AVAILABLE:
    print("\n⭐ 4단계: 양자화 인식 훈련(QAT)")
    qat_model = apply_quantization_aware_training(model, inputs_train, outputs_train, inputs_val, outputs_val)
    
    qat_int8_path = os.path.join(output_dir, 'qat_int8.tflite')
    qat_tflite_model = convert_qat_to_int8(qat_model, inputs_train, outputs_train, qat_int8_path)
    qat_results = quick_model_check(qat_int8_path, inputs_test, outputs_test, "QAT INT8")
    results_summary['QAT INT8'] = qat_results['accuracy']
else:
    print("\n⚠️ 4단계: QAT 라이브러리 없음 - 건너뜀")

# 5단계: 초간단 모델로 시도 (성능이 낮을 때)
best_int8_accuracy = max([results_summary.get('개선된 INT8', 0), results_summary.get('최고 성능 INT8', 0)])
if best_int8_accuracy < 0.7:  # 70% 미만이면
    print("\n🔧 5단계: 초간단 모델로 재시도")
    
    # 초간단 모델 훈련
    print("초간단 모델 훈련 중...")
    simple_history = simple_model.fit(
        inputs_train, outputs_train,
        validation_data=(inputs_val, outputs_val),
        epochs=50,
        batch_size=8,
        verbose=0
    )
    
    # 초간단 모델 성능 확인
    simple_pred = simple_model.predict(inputs_test)
    simple_accuracy = np.mean(np.argmax(simple_pred, axis=1) == np.argmax(outputs_test, axis=1))
    print(f"초간단 모델 Float32 정확도: {simple_accuracy*100:.2f}%")
    
    if simple_accuracy > 0.8:  # 80% 이상이면
        simple_int8_path = os.path.join(output_dir, 'simple_int8.tflite')
        simple_int8_model = ultra_enhanced_int8_conversion(simple_model, inputs_train, outputs_train, simple_int8_path)
        simple_results = quick_model_check(simple_int8_path, inputs_test, outputs_test, "초간단 INT8")
        results_summary['초간단 INT8'] = simple_results['accuracy']

print("\n" + "="*60)
print("🎉 양자화 성능 개선 완료!")
print("="*60)

🎯 단계별 양자화 성능 개선 시작!

📊 기준 성능 (Float32): 100.00%

🔄 1단계: 혼합 정밀도 양자화
혼합 정밀도 양자화 변환 중...
INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmp_y03wpxf\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmp_y03wpxf\assets


대표 데이터 생성 중... (108개 샘플)
✅ 혼합 정밀도 모델 저장 완료: C:/Users/user/Desktop/tinymlmagicwand\tflite_models\mixed_precision.tflite

🔍 혼합 정밀도 빠른 점검
모델 크기: 18.8KB
입력 타입: <class 'numpy.float32'>
출력 타입: <class 'numpy.float32'>
정확도: 1.0000 (100.00%)
평균 추론 시간: 0.70ms
📊 Float32 (또는 혼합 정밀도)

🎯 2단계: 개선된 INT8 양자화 (클래스별 균등 샘플링)
개선된 INT8 양자화 변환 중...
INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpin_0l3f3\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpin_0l3f3\assets


포괄적 대표 데이터 생성 중... (클래스별 균등 샘플링)
✅ 개선된 INT8 모델 저장 완료: C:/Users/user/Desktop/tinymlmagicwand\tflite_models\enhanced_int8.tflite

🔍 개선된 INT8 빠른 점검
모델 크기: 18.5KB
입력 타입: <class 'numpy.int8'>
출력 타입: <class 'numpy.int8'>
정확도: 0.5217 (52.17%)
평균 추론 시간: 0.61ms
✅ 완전 INT8 양자화

🚀 3단계: 최고 성능 INT8 양자화 (전체 데이터 사용)
최고 성능 INT8 양자화 변환 중...
INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpy3ov0anp\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpy3ov0anp\assets


최고 성능 대표 데이터 생성 중... (전체 훈련 데이터 사용)
✅ 최고 성능 INT8 모델 저장 완료: C:/Users/user/Desktop/tinymlmagicwand\tflite_models\ultra_int8.tflite

🔍 최고 성능 INT8 빠른 점검
모델 크기: 18.5KB
입력 타입: <class 'numpy.int8'>
출력 타입: <class 'numpy.int8'>
정확도: 0.5217 (52.17%)
평균 추론 시간: 0.70ms
✅ 완전 INT8 양자화

⚠️ 4단계: QAT 라이브러리 없음 - 건너뜀

🔧 5단계: 초간단 모델로 재시도
초간단 모델 훈련 중...
초간단 모델 Float32 정확도: 95.65%
최고 성능 INT8 양자화 변환 중...
INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpk86qfn96\assets


INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpk86qfn96\assets


최고 성능 대표 데이터 생성 중... (전체 훈련 데이터 사용)
✅ 최고 성능 INT8 모델 저장 완료: C:/Users/user/Desktop/tinymlmagicwand\tflite_models\simple_int8.tflite

🔍 초간단 INT8 빠른 점검
모델 크기: 4.3KB
입력 타입: <class 'numpy.int8'>
출력 타입: <class 'numpy.int8'>
정확도: 0.1739 (17.39%)
평균 추론 시간: 0.07ms
✅ 완전 INT8 양자화

🎉 양자화 성능 개선 완료!


In [61]:
# 📊 최종 결과 비교 및 최적 모델 선택
print("\n📊 최종 성능 비교:")
print("-" * 40)

best_model_name = ""
best_accuracy = 0
best_model_path = ""

for model_name, accuracy in results_summary.items():
    print(f"{model_name:15}: {accuracy*100:6.2f}%")
    
    # INT8 모델 중 최고 성능 찾기
    if 'INT8' in model_name and accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = model_name
        
        # 모델 경로 매핑
        if model_name == '개선된 INT8':
            best_model_path = enhanced_int8_path
        elif model_name == '최고 성능 INT8':
            best_model_path = ultra_int8_path
        elif model_name == 'QAT INT8':
            best_model_path = qat_int8_path
        elif model_name == '초간단 INT8':
            best_model_path = simple_int8_path

print("-" * 40)

# 최적 모델 정보
if best_model_name:
    print(f"\n🏆 최고 성능 INT8 모델: {best_model_name}")
    print(f"   정확도: {best_accuracy*100:.2f}%")
    print(f"   파일: {best_model_path}")
    
    # 최적 모델을 메인 모델로 복사
    main_model_path = os.path.join(output_dir, 'gesture_model_int8_best.tflite')
    import shutil
    shutil.copy2(best_model_path, main_model_path)
    print(f"   ✅ 최적 모델 복사: {main_model_path}")
else:
    print("\n⚠️ INT8 모델 성능이 모두 낮습니다. 혼합 정밀도 모델 사용을 권장합니다.")
    main_model_path = mixed_precision_path

# 성능 개선 요약
original_int8_accuracy = 0.17  # 원래 성능 (예시)
improvement = (best_accuracy - original_int8_accuracy) * 100

print(f"\n📈 성능 개선 요약:")
print(f"   원래 INT8 성능: ~17%")
print(f"   개선된 성능: {best_accuracy*100:.2f}%")
print(f"   개선도: +{improvement:.1f}%p")

if best_accuracy > 0.8:
    print("   🎉 실용적 수준 달성!")
elif best_accuracy > 0.6:
    print("   ✅ 상당한 개선 달성!")
else:
    print("   ⚠️ 추가 개선 필요")


📊 최종 성능 비교:
----------------------------------------
Float32        : 100.00%
혼합 정밀도         : 100.00%
개선된 INT8       :  52.17%
최고 성능 INT8     :  52.17%
----------------------------------------

🏆 최고 성능 INT8 모델: 개선된 INT8
   정확도: 52.17%
   파일: C:/Users/user/Desktop/tinymlmagicwand\tflite_models\enhanced_int8.tflite
   ✅ 최적 모델 복사: C:/Users/user/Desktop/tinymlmagicwand\tflite_models\gesture_model_int8_best.tflite

📈 성능 개선 요약:
   원래 INT8 성능: ~17%
   개선된 성능: 52.17%
   개선도: +35.2%p
   ⚠️ 추가 개선 필요


TypeError: comprehensive_int8_check() missing 1 required positional argument: 'ultra_int8_path'

## 7. Arduino 배포 가이드

최적화된 TFLite 모델을 Arduino에 배포하기 위한 가이드입니다.

In [52]:
def generate_arduino_header(model_path, header_path):
    """
    TFLite 모델을 Arduino용 C++ 헤더 파일로 변환
    """
    with open(model_path, 'rb') as f:
        model_data = f.read()
    
    header_content = f"""// 자동 생성된 TensorFlow Lite 모델 헤더 파일
// 모델 크기: {len(model_data)} bytes
// 최적화: {best_model_name if best_model_name else '혼합 정밀도'}
// 정확도: {best_accuracy*100:.2f}%

#ifndef GESTURE_MODEL_H
#define GESTURE_MODEL_H

const unsigned char gesture_model_tflite[] = {{
"""
    
    # 바이트 데이터를 16진수로 변환
    for i, byte in enumerate(model_data):
        if i % 16 == 0:
            header_content += "\n  "
        header_content += f"0x{byte:02x}"
        if i < len(model_data) - 1:
            header_content += ", "
    
    header_content += f"""
}};

const unsigned int gesture_model_tflite_len = {len(model_data)};

#endif  // GESTURE_MODEL_H
"""
    
    with open(header_path, 'w') as f:
        f.write(header_content)
    
    print(f"✅ Arduino 헤더 파일 생성 완료: {header_path}")
    print(f"   모델 크기: {len(model_data)} bytes")
    return len(model_data)

# Arduino 헤더 파일 생성
header_path = os.path.join(output_dir, 'gesture_model.h')
model_size_bytes = generate_arduino_header(main_model_path, header_path)

✅ Arduino 헤더 파일 생성 완료: C:/Users/user/Desktop/tinymlmagicwand\tflite_models\gesture_model.h
   모델 크기: 18992 bytes


In [69]:
import os
import shutil

def generate_arduino_header_multi(model_path, header_path, model_name, accuracy, model_type):
    """
    TFLite 모델을 Arduino용 C++ 헤더 파일로 변환 (개선된 버전)
    """
    with open(model_path, 'rb') as f:
        model_data = f.read()
    
    # 변수명을 모델 타입에 따라 다르게 설정
    var_name = f"gesture_model_{model_type.lower()}_tflite"
    len_name = f"gesture_model_{model_type.lower()}_tflite_len"
    
    header_content = f"""// 자동 생성된 TensorFlow Lite 모델 헤더 파일
// 모델명: {model_name}
// 모델 타입: {model_type}
// 모델 크기: {len(model_data)} bytes ({len(model_data)/1024:.1f}KB)
// 정확도: {accuracy:.2f}%
// 생성일: {time.strftime("%Y-%m-%d %H:%M:%S")}

#ifndef GESTURE_MODEL_{model_type.upper()}_H
#define GESTURE_MODEL_{model_type.upper()}_H

const unsigned char {var_name}[] = {{
"""
    
    # 바이트 데이터를 16진수로 변환
    for i, byte in enumerate(model_data):
        if i % 16 == 0:
            header_content += "\n  "
        header_content += f"0x{byte:02x}"
        if i < len(model_data) - 1:
            header_content += ", "
    
    header_content += f"""
}};

const unsigned int {len_name} = {len(model_data)};

#endif  // GESTURE_MODEL_{model_type.upper()}_H
"""
    
    with open(header_path, 'w') as f:
        f.write(header_content)
    
    print(f"✅ {model_name} 헤더 파일 생성 완료: {header_path}")
    print(f"   모델 크기: {len(model_data)} bytes ({len(model_data)/1024:.1f}KB)")
    print(f"   정확도: {accuracy:.2f}%")
    
    return len(model_data)

# 🎯 3개 모델 정보 정의
models_info = [
    {
        'name': '혼합 정밀도 모델',
        'path': 'C:/Users/user/Desktop/tinymlmagicwand/tflite_models/mixed_precision.tflite',
        'accuracy': 100.00,
        'type': 'MIXED',
        'header_name': 'gesture_model_mixed.h'
    },
    {
        'name': '개선된 INT8 모델', 
        'path': 'C:/Users/user/Desktop/tinymlmagicwand/tflite_models/enhanced_int8.tflite',
        'accuracy': 52.17,
        'type': 'INT8',
        'header_name': 'gesture_model_int8.h'
    },
    {
        'name': '초간단 INT8 모델',
        'path': 'C:/Users/user/Desktop/tinymlmagicwand/tflite_models/simple_int8.tflite', 
        'accuracy': 17.39,
        'type': 'SIMPLE',
        'header_name': 'gesture_model_simple.h'
    }
]

# 🚀 모든 모델을 C++ 헤더로 변환
print("🔧 3개 모델을 C++ 헤더 파일로 변환 중...")
print("=" * 60)

output_dir = 'C:/Users/user/Desktop/tinymlmagicwand/tflite_models'
model_sizes = {}

for model_info in models_info:
    print(f"\\n📄 {model_info['name']} 변환 중...")
    
    model_path = model_info['path']
    header_path = os.path.join(output_dir, model_info['header_name'])
    
    # 파일 존재 확인
    if os.path.exists(model_path):
        size = generate_arduino_header_multi(
            model_path=model_path,
            header_path=header_path,
            model_name=model_info['name'],
            accuracy=model_info['accuracy'],
            model_type=model_info['type']
        )
        model_sizes[model_info['name']] = size
    else:
        print(f"❌ 파일을 찾을 수 없습니다: {model_path}")

print("\\n" + "=" * 60)
print("🎉 모든 모델 변환 완료!")
print("=" * 60)

# 📊 결과 요약
print("\\n📊 생성된 헤더 파일 요약:")
print("-" * 50)
for model_info in models_info:
    if model_info['name'] in model_sizes:
        print(f"{model_info['name']:20}: {model_info['header_name']}")
        print(f"{'':20}  크기: {model_sizes[model_info['name']]/1024:.1f}KB")
        print(f"{'':20}  정확도: {model_info['accuracy']:.2f}%")
        print()

print("📁 모든 파일 위치:")
print(f"   {output_dir}")

🔧 3개 모델을 C++ 헤더 파일로 변환 중...
\n📄 혼합 정밀도 모델 변환 중...
✅ 혼합 정밀도 모델 헤더 파일 생성 완료: C:/Users/user/Desktop/tinymlmagicwand/tflite_models\gesture_model_mixed.h
   모델 크기: 19208 bytes (18.8KB)
   정확도: 100.00%
\n📄 개선된 INT8 모델 변환 중...
✅ 개선된 INT8 모델 헤더 파일 생성 완료: C:/Users/user/Desktop/tinymlmagicwand/tflite_models\gesture_model_int8.h
   모델 크기: 18992 bytes (18.5KB)
   정확도: 52.17%
\n📄 초간단 INT8 모델 변환 중...
✅ 초간단 INT8 모델 헤더 파일 생성 완료: C:/Users/user/Desktop/tinymlmagicwand/tflite_models\gesture_model_simple.h
   모델 크기: 4384 bytes (4.3KB)
   정확도: 17.39%
\n============================================================
🎉 모든 모델 변환 완료!
\n📊 생성된 헤더 파일 요약:
--------------------------------------------------
혼합 정밀도 모델           : gesture_model_mixed.h
                      크기: 18.8KB
                      정확도: 100.00%

개선된 INT8 모델         : gesture_model_int8.h
                      크기: 18.5KB
                      정확도: 52.17%

초간단 INT8 모델         : gesture_model_simple.h
                      크기: 4.3KB
                

In [70]:
# 🎉 최종 요약 및 파일 목록
print("\n" + "="*70)
print("🎉 제스처 인식 TinyML 파이프라인 완료! (개선된 버전)")
print("="*70)

print(f"\n📁 생성된 파일들 (경로: {output_dir}):")
print(f"   📄 gesture_model_int8_best.tflite - 최적 INT8 모델")
print(f"   📄 mixed_precision.tflite - 혼합 정밀도 모델")
print(f"   📄 gesture_model.h - Arduino 헤더 파일")
print(f"   📄 gesture_recognition_improved.ino - 개선된 Arduino 코드")
print(f"   📄 deployment_guide_improved.md - 상세 배포 가이드")

if best_model_name:
    print(f"\n🏆 최고 성능 모델:")
    print(f"   🎯 모델: {best_model_name}")
    print(f"   📊 정확도: {best_accuracy:.4f} ({best_accuracy*100:.2f}%)")
    print(f"   💾 크기: {model_size_bytes/1024:.1f}KB")
    print(f"   ⚡ 메모리: {max(8192, model_size_bytes * 2)/1024:.1f}KB")

print(f"\n📈 성능 개선 요약:")
improvement = (best_accuracy - 0.17) * 100  # 원래 17%에서 개선
print(f"   원래 성능: ~17%")
print(f"   개선된 성능: {best_accuracy*100:.2f}%")
print(f"   개선도: +{improvement:.1f}%p")

if best_accuracy > 0.8:
    status = "🎉 실용적 수준 달성!"
elif best_accuracy > 0.6:
    status = "✅ 상당한 개선 달성!"
else:
    status = "⚠️ 혼합 정밀도 모델 사용 권장"

print(f"   상태: {status}")

print(f"\n🚀 다음 단계:")
print(f"   1. Arduino IDE에서 gesture_recognition_improved.ino 열기")
print(f"   2. gesture_model.h 파일을 같은 폴더에 복사")
print(f"   3. 필요한 라이브러리 설치 (LSM9DS1, TensorFlowLite)")
print(f"   4. Arduino Nano 33 BLE에 업로드")
print(f"   5. 시리얼 모니터에서 테스트 ('s' 입력)")

print(f"\n💡 권장사항:")
if best_accuracy > 0.7:
    print(f"   ✅ INT8 모델 사용 가능 - 최고 효율성")
else:
    print(f"   📊 혼합 정밀도 모델 사용 권장 - 높은 정확도")

print(f"   🎯 제스처를 크고 명확하게 수행하세요")
print(f"   📖 상세한 사용법은 deployment_guide_improved.md 참조")

print(f"\n✨ 개선된 TinyML 모델이 성공적으로 준비되었습니다!")
print(f"   데이터 경로: {DATA_ROOT}")
print(f"   모델 경로: {output_dir}")


🎉 제스처 인식 TinyML 파이프라인 완료! (개선된 버전)

📁 생성된 파일들 (경로: C:/Users/user/Desktop/tinymlmagicwand/tflite_models):
   📄 gesture_model_int8_best.tflite - 최적 INT8 모델
   📄 mixed_precision.tflite - 혼합 정밀도 모델
   📄 gesture_model.h - Arduino 헤더 파일
   📄 gesture_recognition_improved.ino - 개선된 Arduino 코드
   📄 deployment_guide_improved.md - 상세 배포 가이드

🏆 최고 성능 모델:
   🎯 모델: 개선된 INT8
   📊 정확도: 0.5217 (52.17%)
   💾 크기: 18.5KB
   ⚡ 메모리: 37.1KB

📈 성능 개선 요약:
   원래 성능: ~17%
   개선된 성능: 52.17%
   개선도: +35.2%p
   상태: ⚠️ 혼합 정밀도 모델 사용 권장

🚀 다음 단계:
   1. Arduino IDE에서 gesture_recognition_improved.ino 열기
   2. gesture_model.h 파일을 같은 폴더에 복사
   3. 필요한 라이브러리 설치 (LSM9DS1, TensorFlowLite)
   4. Arduino Nano 33 BLE에 업로드
   5. 시리얼 모니터에서 테스트 ('s' 입력)

💡 권장사항:
   📊 혼합 정밀도 모델 사용 권장 - 높은 정확도
   🎯 제스처를 크고 명확하게 수행하세요
   📖 상세한 사용법은 deployment_guide_improved.md 참조

✨ 개선된 TinyML 모델이 성공적으로 준비되었습니다!
   데이터 경로: C:/Users/user/Desktop/tinymlmagicwand
   모델 경로: C:/Users/user/Desktop/tinymlmagicwand/tflite_models
